In [3]:
import pyexasol
import pandas as pd
import boto3
import time
import io
import numpy as np
from matplotlib import pyplot as plt


dwh_host = '***'
dwh_port = '8563'
dwh_user = '***'
dwh_pass = '***'

#connect to db
dwh_conn = pyexasol.connect(dsn=dwh_host + ':' + dwh_port, user=dwh_user, password=dwh_pass)

## data function to get iterator data

In [7]:
#get iterator month data
def get_feature_types():

    #get iterator datam
    
    v_sql = """
SELECT distinct
        feature_type
from
    betting_mart.ml_poisson_features
where
    full_time_home_goals is not null
    and home_shots_for_ema is not null
    and away_shots_against_ema is not null
    """
    
    return dwh_conn.export_to_pandas(v_sql)


#get iterator month data
def get_iterator_data(feature_type_filter):

    #get iterator datam
    
    v_sql = """SELECT distinct
        to_char(match_date,'yyyymm') iterator
    FROM 
        betting_mart.ml_poisson_features
    WHERE
        --2014_15 season dropped for stable features
        season <> '2014_2015'
        --2015_16 is first training season
        AND season <> '2015_2016' and
        --feature type filter
        feature_type = '""" + str(feature_type_filter) + """' and
        --ohne null values
        home_goals_for_ema is not null and
        home_goals_against_ema is not null and
        away_goals_for_ema is not null and
        away_goals_against_ema is not null and
        home_xg_for_ema is not null and
        home_xg_against_ema is not null and
        away_xg_for_ema is not null and
        away_xg_against_ema is not null and
        home_shots_for_ema is not null and
        home_shots_against_ema is not null and
        away_shots_for_ema is not null and
        away_shots_against_ema is not null and
        home_sot_for_ema is not null and
        home_sot_against_ema is not null and
        away_sot_for_ema is not null and
        away_sot_against_ema is not null and
        home_corners_for_ema is not null and
        home_corners_against_ema is not null and
        away_corners_for_ema is not null and
        away_corners_against_ema is not null and
        home_deep_for_ema is not null and
        home_deep_against_ema is not null and
        away_deep_for_ema is not null and
        away_deep_against_ema is not null and
        home_ppda_for_ema is not null and
        home_ppda_against_ema is not null and
        away_ppda_for_ema is not null and
        away_ppda_against_ema is not null and
        full_time_home_goals is not null and
        full_time_away_goals is not null
    ORDER BY 1"""

    return dwh_conn.export_to_pandas(v_sql)


def get_training_data(feature_type_filter, month_filter):

    v_sql = """
select
    round(home_goals_for_ema,1)            home_goals_for_ema,
    round(home_goals_against_ema,1)        home_goals_against_ema,
    round(away_goals_for_ema,1)            away_goals_for_ema,
    round(away_goals_against_ema,1)        away_goals_against_ema,
    round(home_xg_for_ema,1)               home_xg_for_ema,
    round(home_xg_against_ema,1)           home_xg_against_ema,
    round(away_xg_for_ema,1)               away_xg_for_ema,
    round(away_xg_against_ema,1)           away_xg_against_ema,
    round(home_shots_for_ema,1)               home_shots_for_ema,
    round(home_shots_against_ema,1)           home_shots_against_ema,
    round(away_shots_for_ema,1)               away_shots_for_ema,
    round(away_shots_against_ema,1)           away_shots_against_ema,
    round(home_sot_for_ema,1)               home_sot_for_ema,
    round(home_sot_against_ema,1)           home_sot_against_ema,
    round(away_sot_for_ema,1)               away_sot_for_ema,
    round(away_sot_against_ema,1)           away_sot_against_ema,
    round(home_corners_for_ema,1)               home_corners_for_ema,
    round(home_corners_against_ema,1)           home_corners_against_ema,
    round(away_corners_for_ema,1)               away_corners_for_ema,
    round(away_corners_against_ema,1)           away_corners_against_ema,
    round(home_deep_for_ema,1)               home_deep_for_ema,
    round(home_deep_against_ema,1)           home_deep_against_ema,
    round(away_deep_for_ema,1)               away_deep_for_ema,
    round(away_deep_against_ema,1)           away_deep_against_ema,
    round(home_ppda_for_ema,1)               home_ppda_for_ema,
    round(home_ppda_against_ema,1)           home_ppda_against_ema,
    round(away_ppda_for_ema,1)               away_ppda_for_ema,
    round(away_ppda_against_ema,1)           away_ppda_against_ema,
    full_time_home_goals               full_time_home_goals,
    full_time_away_goals               full_time_away_goals
from
    betting_mart.ml_poisson_features
where
    --filter for iterator
    to_char(match_date,'yyyymm') < """ + str(month_filter) + """ and
    feature_type = '""" + str(feature_type_filter) + """' and
    home_goals_for_ema is not null and
    home_goals_against_ema is not null and
    away_goals_for_ema is not null and
    away_goals_against_ema is not null and
    home_xg_for_ema is not null and
    home_xg_against_ema is not null and
    away_xg_for_ema is not null and
    away_xg_against_ema is not null and
    home_shots_for_ema is not null and
    home_shots_against_ema is not null and
    away_shots_for_ema is not null and
    away_shots_against_ema is not null and
    home_sot_for_ema is not null and
    home_sot_against_ema is not null and
    away_sot_for_ema is not null and
    away_sot_against_ema is not null and
    home_corners_for_ema is not null and
    home_corners_against_ema is not null and
    away_corners_for_ema is not null and
    away_corners_against_ema is not null and
    home_deep_for_ema is not null and
    home_deep_against_ema is not null and
    away_deep_for_ema is not null and
    away_deep_against_ema is not null and
    home_ppda_for_ema is not null and
    home_ppda_against_ema is not null and
    away_ppda_for_ema is not null and
    away_ppda_against_ema is not null and
    full_time_home_goals is not null and
    full_time_away_goals is not null
    """
    
    
    return dwh_conn.export_to_pandas(v_sql)



def get_prediction_data(feature_type_filter, month_filter):
    
    v_sql = """
select
    football_match_his_lid,
    round(home_goals_for_ema,1)            home_goals_for_ema,
    round(home_goals_against_ema,1)        home_goals_against_ema,
    round(away_goals_for_ema,1)            away_goals_for_ema,
    round(away_goals_against_ema,1)        away_goals_against_ema,
    round(home_xg_for_ema,1)               home_xg_for_ema,
    round(home_xg_against_ema,1)           home_xg_against_ema,
    round(away_xg_for_ema,1)               away_xg_for_ema,
    round(away_xg_against_ema,1)           away_xg_against_ema,
    round(home_shots_for_ema,1)               home_shots_for_ema,
    round(home_shots_against_ema,1)           home_shots_against_ema,
    round(away_shots_for_ema,1)               away_shots_for_ema,
    round(away_shots_against_ema,1)           away_shots_against_ema,
    round(home_sot_for_ema,1)               home_sot_for_ema,
    round(home_sot_against_ema,1)           home_sot_against_ema,
    round(away_sot_for_ema,1)               away_sot_for_ema,
    round(away_sot_against_ema,1)           away_sot_against_ema,
    round(home_corners_for_ema,1)               home_corners_for_ema,
    round(home_corners_against_ema,1)           home_corners_against_ema,
    round(away_corners_for_ema,1)               away_corners_for_ema,
    round(away_corners_against_ema,1)           away_corners_against_ema,
    round(home_deep_for_ema,1)               home_deep_for_ema,
    round(home_deep_against_ema,1)           home_deep_against_ema,
    round(away_deep_for_ema,1)               away_deep_for_ema,
    round(away_deep_against_ema,1)           away_deep_against_ema,
    round(home_ppda_for_ema,1)               home_ppda_for_ema,
    round(home_ppda_against_ema,1)           home_ppda_against_ema,
    round(away_ppda_for_ema,1)               away_ppda_for_ema,
    round(away_ppda_against_ema,1)           away_ppda_against_ema
from
    betting_mart.ml_poisson_features
where
    --filter for iterator
    to_char(match_date,'yyyymm') = """ + str(month_filter) + """ and
    feature_type = '""" + str(feature_type_filter) + """' and
    home_goals_for_ema is not null and
    home_goals_against_ema is not null and
    away_goals_for_ema is not null and
    away_goals_against_ema is not null and
    home_xg_for_ema is not null and
    home_xg_against_ema is not null and
    away_xg_for_ema is not null and
    away_xg_against_ema is not null and
    home_shots_for_ema is not null and
    home_shots_against_ema is not null and
    away_shots_for_ema is not null and
    away_shots_against_ema is not null and
    home_sot_for_ema is not null and
    home_sot_against_ema is not null and
    away_sot_for_ema is not null and
    away_sot_against_ema is not null and
    home_corners_for_ema is not null and
    home_corners_against_ema is not null and
    away_corners_for_ema is not null and
    away_corners_against_ema is not null and
    home_deep_for_ema is not null and
    home_deep_against_ema is not null and
    away_deep_for_ema is not null and
    away_deep_against_ema is not null and
    home_ppda_for_ema is not null and
    home_ppda_against_ema is not null and
    away_ppda_for_ema is not null and
    away_ppda_against_ema is not null
    """
    
    return dwh_conn.export_to_pandas(v_sql)

## data function to get home and away data

In [9]:

import pandas
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor


#truncate result table
#dwh_conn.execute('truncate table BETTING_DV.football_match_his_l_s_ml_poisson_expected_goals')


#get different feature types
df_feature_iterator = get_feature_types()

for index, row in df_feature_iterator.iterrows():
    print('Loop for feature type: ' + str(row['FEATURE_TYPE']))
    
    print('...getting data monthly loops for simulation')
    
    df_month_iterator = get_iterator_data(row['FEATURE_TYPE'])
    
    for index2, row2 in df_month_iterator.iterrows():
        
        print('...simulation for month ' + str(row2['ITERATOR']))
        
        print('.....getting training data')
        
        #get data to train home expGoals model
        df_train_data = get_training_data(row['FEATURE_TYPE'], row2['ITERATOR'])
    
        #split features and pred classes
        df_train_x = df_train_data.iloc[:,:28]
        df_train_home_y = df_train_data.iloc[:,28]
        df_train_away_y = df_train_data.iloc[:,29]
                
        print('......training models')
        
        print('......train_x: ' + str(df_train_x.shape))
        print('......train home y: ' + str(df_train_home_y.shape))
        print('......train away y: ' + str(df_train_away_y.shape))
        
        
        model_home = XGBRegressor(
                                            n_estimators = 100,
                                            max_depth = 3,
                                            eta = 0.06
                                            ).fit(df_train_x, df_train_home_y)
        
        
        model_away = XGBRegressor(
                                            n_estimators = 100,
                                            max_depth = 2,
                                            eta = 0.06
                                            ).fit(df_train_x, df_train_away_y)


        print('......getting prediction data')
        
        #get data to train home expGoals model
        df_pred_data = get_prediction_data(row['FEATURE_TYPE'], row2['ITERATOR'])
        
        print('......df_pred_data: ' + str(df_pred_data.shape))
        
        #do prediction
        df_pred_x = df_pred_data.iloc[:,1:29]
        
        print('......executing predition')  
    
        df_pred_home_expGoals = model_home.predict(df_pred_x)
        df_pred_away_expGoals = model_away.predict(df_pred_x)
    
        #create result data frame
        df_result_data = pd.DataFrame(df_pred_data.iloc[:,0:1])
        df_result_data['feature_type'] = 'xgboost/full/' + str(row['FEATURE_TYPE'])
        df_result_data['home_exp_goals'] = df_pred_home_expGoals
        df_result_data['away_exp_goals'] = df_pred_away_expGoals
        df_result_data['interator_param'] = str(row2['ITERATOR'])


        print(df_result_data.head())
        
        print('......data gets written to the db')    
        dwh_conn.import_from_pandas(df_result_data,('BETTING_DV','football_match_his_l_s_ml_poisson_expected_goals'))
    

Loop for feature type: EMA15 COMB
...getting data monthly loops for simulation
...simulation for month 201608
.....getting training data
......training models
......train_x: (3590, 28)
......train home y: (3590,)
......train away y: (3590,)
......getting prediction data
......df_pred_data: (100, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID             feature_type  home_exp_goals  \
0  df74d6019d8ec8969da3b8a2d316d111  xgboost/full/EMA15 COMB        1.336190   
1  733d356d163a75c04365593496d232e1  xgboost/full/EMA15 COMB        1.059884   
2  acee56b53a1a434affc5cf8caedd2ef1  xgboost/full/EMA15 COMB        1.083414   
3  6b2f710e0f9b6817cee287a160cfa86f  xgboost/full/EMA15 COMB        1.468203   
4  fe382c3ac79fce6472a8d35927482d7f  xgboost/full/EMA15 COMB        1.266825   

   away_exp_goals interator_param  
0        0.945688          201608  
1        1.281373          201608  
2        1.072639          201608  
3        1.240382          201608  
4        1.2

...simulation for month 201705
.....getting training data
......training models
......train_x: (5240, 28)
......train home y: (5240,)
......train away y: (5240,)
......getting prediction data
......df_pred_data: (167, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID             feature_type  home_exp_goals  \
0  3d76709204343d71cb161740e78c64c1  xgboost/full/EMA15 COMB        1.777964   
1  9b0f03d0e1aedb7ffb78237c59263e3c  xgboost/full/EMA15 COMB        1.532905   
2  aaa293e30ea74fcf1e9e6d1526121c3b  xgboost/full/EMA15 COMB        0.782912   
3  f227a728ef6dfd82a1385e8d5c5a98ee  xgboost/full/EMA15 COMB        1.543083   
4  accc5bb1186c5fad753e29fb51bb8dc3  xgboost/full/EMA15 COMB        1.399579   

   away_exp_goals interator_param  
0        0.842479          201705  
1        1.242779          201705  
2        2.066784          201705  
3        1.076760          201705  
4        0.952254          201705  
......data gets written to the db
...simulation for mon

...simulation for month 201804
.....getting training data
......training models
......train_x: (6851, 28)
......train home y: (6851,)
......train away y: (6851,)
......getting prediction data
......df_pred_data: (239, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID             feature_type  home_exp_goals  \
0  746abc9074588653c70730df0b0b757a  xgboost/full/EMA15 COMB        1.345997   
1  efe7ef407a2d5e28b573ec2406e6c804  xgboost/full/EMA15 COMB        1.338189   
2  c5894cfc0460ead0d141f7f339479ee6  xgboost/full/EMA15 COMB        1.588272   
3  4cffde082e2ad10df7c31c66a4244591  xgboost/full/EMA15 COMB        3.061105   
4  32602de76aabdad2086212de5986a994  xgboost/full/EMA15 COMB        2.010361   

   away_exp_goals interator_param  
0        1.128420          201804  
1        0.918212          201804  
2        0.955048          201804  
3        0.604199          201804  
4        0.990698          201804  
......data gets written to the db
...simulation for mon

...simulation for month 201903
.....getting training data
......training models
......train_x: (8460, 28)
......train home y: (8460,)
......train away y: (8460,)
......getting prediction data
......df_pred_data: (186, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID             feature_type  home_exp_goals  \
0  bc5d683ea6d354e31d9eb5bad57265af  xgboost/full/EMA15 COMB        0.949078   
1  3c7f2292d9823ec30f5b9e24996d8165  xgboost/full/EMA15 COMB        1.310699   
2  582430e38c10da9d75a11e77c45e5c93  xgboost/full/EMA15 COMB        1.387318   
3  0e101323a19903b806f1abd41e6289e3  xgboost/full/EMA15 COMB        1.411323   
4  3557b77be42f233c24ecfe43a4100cac  xgboost/full/EMA15 COMB        1.223335   

   away_exp_goals interator_param  
0        1.744381          201903  
1        1.154611          201903  
2        1.058844          201903  
3        1.459421          201903  
4        1.501422          201903  
......data gets written to the db
...simulation for mon

...simulation for month 202002
.....getting training data
......training models
......train_x: (10078, 28)
......train home y: (10078,)
......train away y: (10078,)
......getting prediction data
......df_pred_data: (215, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID             feature_type  home_exp_goals  \
0  145038a9dee8ff0142eb361cd344b9a4  xgboost/full/EMA15 COMB        1.145313   
1  3dcc3a7ad07b084842289ac67af46211  xgboost/full/EMA15 COMB        1.442973   
2  e17d31da8fc18304f217959577d6a230  xgboost/full/EMA15 COMB        1.449677   
3  e4c4935cbbeeb9afd208b732718add26  xgboost/full/EMA15 COMB        1.403168   
4  de075b4298007499c589c12e130a0178  xgboost/full/EMA15 COMB        2.151950   

   away_exp_goals interator_param  
0        1.346451          202002  
1        1.528538          202002  
2        1.326254          202002  
3        1.278966          202002  
4        0.805237          202002  
......data gets written to the db
...simulation for 

...

...simulation for month 201805
.....getting training data
......training models
......train_x: (7090, 28)
......train home y: (7090,)
......train away y: (7090,)
......getting prediction data
......df_pred_data: (136, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID             feature_type  home_exp_goals  \
0  8dabd32adc5381853616cfed1a038f6a  xgboost/full/EMA20 COMB        1.064623   
1  a2495ade9cf8ed85111c99a2b570bdfc  xgboost/full/EMA20 COMB        1.347225   
2  101f7ec4a9522fd777065f9103e55e5f  xgboost/full/EMA20 COMB        3.283803   
3  65f3eb1f7b177501019b05dfbf184e73  xgboost/full/EMA20 COMB        1.075379   
4  e2318d269d633ebbfbb13f5018450b64  xgboost/full/EMA20 COMB        1.445012   

   away_exp_goals interator_param  
0        1.157560          201805  
1        0.947394          201805  
2        0.571980          201805  
3        1.133842          201805  
4        1.366248          201805  
......data gets written to the db
...simulation for mon

...simulation for month 201904
.....getting training data
......training models
......train_x: (8646, 28)
......train home y: (8646,)
......train away y: (8646,)
......getting prediction data
......df_pred_data: (238, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID             feature_type  home_exp_goals  \
0  fac3a779b1fb8fadf7731af4f59ea0fb  xgboost/full/EMA20 COMB        1.171246   
1  7a94f38f69d5b18ce933d17b93935f35  xgboost/full/EMA20 COMB        2.328357   
2  8544df7977ba890aff78abf35a7c9e1b  xgboost/full/EMA20 COMB        1.172773   
3  1ca759c47d31938ae0d1a794938ce6e1  xgboost/full/EMA20 COMB        2.284937   
4  2224a062d4127e0c6604501815a63d3a  xgboost/full/EMA20 COMB        1.754530   

   away_exp_goals interator_param  
0        1.931113          201904  
1        1.019726          201904  
2        2.165370          201904  
3        0.931742          201904  
4        0.783335          201904  
......data gets written to the db
...simulation for mon

...simulation for month 202003
.....getting training data
......training models
......train_x: (10293, 28)
......train home y: (10293,)
......train away y: (10293,)
......getting prediction data
......df_pred_data: (60, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID             feature_type  home_exp_goals  \
0  98da06a8dcb879de42d7df0ddac017f8  xgboost/full/EMA20 COMB        1.112360   
1  7dfb49ae68215726f5e4cefca04abf59  xgboost/full/EMA20 COMB        2.826492   
2  2e4a8f62723e5cdec91e35a3d61bfdd8  xgboost/full/EMA20 COMB        1.410924   
3  f12efde5297ebf3f2d4aa63b4e54085f  xgboost/full/EMA20 COMB        1.519614   
4  e5e88e433e207e605bb186acabd99386  xgboost/full/EMA20 COMB        1.265193   

   away_exp_goals interator_param  
0        1.218633          202003  
1        0.632975          202003  
2        1.071897          202003  
3        1.220456          202003  
4        1.360697          202003  
......data gets written to the db
...simulation for m

...simulation for month 202101
.....getting training data
......training models
......train_x: (11489, 28)
......train home y: (11489,)
......train away y: (11489,)
......getting prediction data
......df_pred_data: (263, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID             feature_type  home_exp_goals  \
0  ee95a9c7954367123e7b957972d4b8c4  xgboost/full/EMA20 COMB        2.552788   
1  4871ec0c74abfcce57b10091eb9404bb  xgboost/full/EMA20 COMB        1.659293   
2  ece4b60f90b4e79aa68ff0e7a1496e4b  xgboost/full/EMA20 COMB        1.662266   
3  af9e1a1576995c07c95114aa03ff11fe  xgboost/full/EMA20 COMB        2.087370   
4  b80e7a62cdaaba8c7f34c7d1a61abecb  xgboost/full/EMA20 COMB        1.449376   

   away_exp_goals interator_param  
0        0.918688          202101  
1        0.943777          202101  
2        1.218238          202101  
3        1.035682          202101  
4        1.146268          202101  
......data gets written to the db
...simulation for 

...simulation for month 202112
.....getting training data
......training models
......train_x: (13263, 28)
......train home y: (13263,)
......train away y: (13263,)
......getting prediction data
......df_pred_data: (208, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID             feature_type  home_exp_goals  \
0  534a973ab80866440ae57c45aec1a8d6  xgboost/full/EMA20 COMB        1.940170   
1  ad8e03622165cd6318bc282fffba7b84  xgboost/full/EMA20 COMB        1.689278   
2  5e02d80f8786802a75957aecdf1830fe  xgboost/full/EMA20 COMB        1.833423   
3  4fa89b28f8608a0391b4d1e36180cc98  xgboost/full/EMA20 COMB        1.756746   
4  cc9a862e4e292b75a347e082adcbe601  xgboost/full/EMA20 COMB        1.564263   

   away_exp_goals interator_param  
0        0.957010          202112  
1        1.406343          202112  
2        0.767862          202112  
3        1.119003          202112  
4        1.510447          202112  
......data gets written to the db
...simulation for 